In [ ]:
pip install git+https://github.com/SKT-AI/KoBART#egg=kobart 

In [ ]:
from transformers import BartModel
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
model = BartModel.from_pretrained(get_pytorch_kobart_model())


In [ ]:

import sys
import requests
import pandas as pd
import numpy as np
import html
from bs4 import BeautifulSoup
import bs4
import re
import json
import time
from datetime import datetime
import ipywidgets

In [ ]:
#네이버 API 아이디 키 값
client_id = "" #1.에서 취득한 아이디 넣기
client_secret = ""  #1. 에서 취득한 키 넣기

In [ ]:

def GetNaverNews(search_word, num= 100):
    # search_word = '부동산' #검색어
    global client_id, client_secret

    encode_type = 'json'  # 출력 방식 json 또는 xml
    max_display = 100  # 출력 뉴스 수
    sort = 'date'  # 결과값의 정렬기준 시간순 date, 관련도 순 sim

    bFirst = True
    for start in range(1, num, 100):
        url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

        # 헤더에 아이디와 키 정보 넣기
        headers = {'X-Naver-Client-Id': client_id,
                   'X-Naver-Client-Secret': client_secret
                   }

        # HTTP요청 보내기
        r = requests.get(url, headers=headers)
        # 요청 결과 보기 200 이면 정상적으로 요청 완료
        if r.status_code == 200:
            if bFirst:
                data = pd.DataFrame(r.json()['items'])
                bFirst = False
            else:
                data = pd.concat([data, pd.DataFrame(r.json()['items'])], axis=0)

    # 태그 삭제
    data['title'] = data['title'].apply(lambda x: re.sub('(<([^>]+)>)', '', html.unescape(x)))
    data['description'] = data['description'].apply(lambda x: re.sub('(<([^>]+)>)', '', html.unescape(x)))

    return data

In [ ]:
data = GetNaverNews('월드컵', num=1000)
data = data.reset_index()
data.head(10)

In [ ]:
words = {}
words_list = np.array([])
for i, row in data.iterrows():
  words[i] = kobart_tokenizer([row['description']], return_tensors='pt')['input_ids']
  words_list = np.append(words_list, words[i])

print(words_list.shape)
words_list = np.unique(words_list)
print(words_list.shape)

In [ ]:
tf_table = np.zeros([data.shape[0], words_list.shape[0]])

for i in range(tf_table.shape[0]):
  for j in range(tf_table.shape[1]):
    tf_table[i, j] = (np.where(words[i]==words_list[j])[1]).shape[0]

  sys.stdout.write('\r[' + str(i) + '/' + str(tf_table.shape[0]) + ']')

tf_table

In [ ]:
df_table = tf_table.copy()
df_table[np.where(df_table>0)] = 1

df_table = df_table.sum(axis=0)
df_table

In [ ]:
idf_table = np.log(data.shape[0] / (1 + df_table))
idf_table

In [ ]:
tf_idf_table = tf_table * idf_table
tf_idf_table

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cossim = cosine_similarity(tf_idf_table, tf_idf_table)

cossim.shape

In [ ]:
from scipy.stats import rankdata

#각 클러스터별 출력한다.

def display_cossim_TFIDF(cnum = 0, top=5) :
  global data, cossim
  print(data.loc[cnum, 'title'])
  print()
  similarity = cossim[cnum,:].copy() #cos유사도 테이블 복사
  # similarity = np.delete(similarity, cnum) #대상 데이터 삭제
  ranks = rankdata(similarity, method='max')
  print(data.loc[np.where(ranks > similarity.shape[0] - top)[0], ['title', 'description']])
  print()
  print(similarity[np.where(ranks > similarity.shape[0] - top)])


ipywidgets.interact(display_cossim_TFIDF, cnum=(0, data.shape[0], 1) , top=(5,20, 1))